In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import requests as rq

In [ ]:
resp = rq.get("https://api.dataforsyningen.dk/autocomplete?q=urtehaven 7 h")

j

In [16]:
resp.json()

[]

In [48]:
import xarray as xr
import datetime as dt
from dash_app.logic.simulation.simulator import Simulator
from dash_app.logic.models import Coordinate, System, SystemParameters, ModuleParameters, InverterParameters, ThermalParameters, Direction
import hvplot.xarray
import hvplot
import pandas as pd
import numpy as np
hvplot.extension("plotly")

In [2]:
nwp = xr.open_dataset("/home/uch/PVForecast/data/etl_function_app/compressed.nc", engine="h5netcdf")

In [3]:
nwp = nwp.drop_vars(["model_run_time_utc"])

In [163]:
def resample_interpolate_center(ds: xr.Dataset, kind: str, accumulate: bool = False, **indexer_kwargs):
    if len(indexer_kwargs) > 1:
        raise ValueError("Center interpolation only supported along one axis.")
    
    dim = list(indexer_kwargs.keys())[0]

    start_freq = pd.Timedelta(f"1{xr.infer_freq(ds[dim])}").to_numpy()

    end_freq = np.timedelta64(list(indexer_kwargs.values())[0])

    ds = ds.copy()
    
    ds = ds.resample(**indexer_kwargs).asfreq()

    ds = ds.interpolate_na(dim=dim, method=kind)

    ds[dim] = ds[dim] - start_freq/2

    return ds

In [164]:
nwp1 = resample_interpolate_center(nwp, time_utc=dt.timedelta(minutes=5), kind="linear")

In [167]:
nwp2 = resample_interpolate_center(nwp, time_utc=dt.timedelta(minutes=5), kind="pchip")

In [168]:
nwp1.isel(x=0, y=0)["global_radiation_W_m2"].hvplot() * nwp2.isel(x=0, y=0)["global_radiation_W_m2"].hvplot()

/home/uch/.pyenv/versions/3.10.8/envs/pvforecast-app/lib/python3.10/site-packages/panel/io/notebook.py:201: BokehUserWarning:

out of range integer may result in loss of precision

/home/uch/.pyenv/versions/3.10.8/envs/pvforecast-app/lib/python3.10/site-packages/panel/io/notebook.py:201: BokehUserWarning:

out of range integer may result in loss of precision



:Overlay
   .Curve.I  :Curve   [time_utc]   (global_radiation_W_m2)
   .Curve.II :Curve   [time_utc]   (global_radiation_W_m2)

In [46]:
nwp_interpolated = nwp.resample(time_utc=dt.timedelta(minutes=5)).interpolate()

In [9]:
coord = Coordinate(
    latitude=56,
    longitude=9,
    altitude=0,
)

In [10]:
direction = Direction(
    azimuth=180,
    elevation=45,
)

In [11]:
system = System(
    system_params=SystemParameters(
        module_params=ModuleParameters(
            temperature_coefficient=0.004,
            dc_capacity=100,
        ),
        inverter_params=InverterParameters(
            nominal_efficiency=0.96,
            ac_capacity=90,
        ),
        thermal_params=ThermalParameters(
            a=-3.47,
            b=-0.0594,
            deltaT=3,
        )
    ),
    direction=direction,
    coordinate=coord,
)

In [12]:
sim = Simulator()

In [16]:
result = sim.run(system, nwp_interpolated)

In [17]:
result["max_power"] = system.system_params.inverter_params.ac_capacity

In [18]:
result[["ac_power", "max_power"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [time_utc]   (value)

In [ ]:
ds = sim.run(system, start_time=dt.datetime(2020,1,1), end_time=dt.datetime(2025,1,2))

In [20]:
ds = ds.load()

In [ ]:
ds.to_dataframe().select_dtypes("float").plot()

In [ ]:
nwp = xr.Dataset(
    data_vars={
        "global_horizontal_irradiance": ("time", [100, 200, 300]),
        "wind_speed_u": ("time", [1, 2, 3]),
        "wind_speed_v": ("time", [1, 2, 3]),
        "air_temperature": ("time", [293.15, 294.15, 295.15]),
    },
    coords={
        "time": [dt.datetime(2020,1,1,12), dt.datetime(2020,1,1,13), dt.datetime(2020,1,1,14)]
    }
)

In [ ]:
coord = Coordinate(
    latitude=52,
    longitude=9,
    altitude=0,
)

In [ ]:
direction = Direction(
    azimuth=180,
    elevation=45,
)

In [ ]:
system = System(
    system_params=SystemParameters(
        module_params=ModuleParameters(
            temperature_coefficient=0.004,
            dc_capacity=100,
        ),
        inverter_params=InverterParameters(
            nominal_efficiency=0.96,
            ac_capacity=90,
        ),
        thermal_params=ThermalParameters(
            a=1,
            b=1,
            deltaT=1,
        )
    ),
    direction=direction,
    coordinate=coord,
)

In [ ]:
simulations.ac_power_from_nwp(system, nwp)

In [ ]:
df = nwp.to_dataframe()

In [ ]:
solar_position = formulas.solar_position_from_time(
    coordinate=coord,
    time=df.index,
)

In [ ]:
wind_speed = formulas.magnitude(df.wind_speed_u, df.wind_speed_v)

In [ ]:
irradiance_components = formulas.estimate_irradiance_components(
    df.global_radiation,
    solar_position["zenith"],
    df.index.dayofyear,
)

In [ ]:
irradiance_components

In [ ]:
ac_power = formulas.ac_power_from_ghi(
    system.system_params,
    df.global_radiation,
    solar_position["zenith"],
    solar_position["azimuth"],
    system.direction.elevation,
    system.direction.azimuth,
    df.index.dayofyear,
    df.air_temperature,
    wind_speed,
)

In [ ]:
ds = nwp.copy()

In [ ]:
ds["ac_power"] = xr.DataArray.from_series(ac_power)

In [ ]:
import xarray as xr
import numpy as np
import datetime as dt

In [ ]:
ds = xr.Dataset(
    data_vars={
        "global_radiation_W_m2": (("time_utc", "x", "y"), np.random.uniform(0, 1000, size=(2, 2, 2))),
        "temperature_K": (("time_utc", "x", "y"), np.random.uniform(263.15, 303.15, size=(2,2,2))),
        "wind_speed_u_m_s": (("time_utc", "x", "y"), np.random.uniform(0, 10, size=(2,2,2))),
        "wind_speed_v_m_s": (("time_utc", "x", "y"), np.random.uniform(0, 10, size=(2,2,2))),
    },
    coords={
        "latitude": (("x", "y"), [[50,60], [51, 61]]),
        "longitude": (("x", "y"), [[70,80], [71, 81]]),
        "time_utc": [dt.datetime(2000,1,1,12), dt.datetime(2000,1,1,13)]
    },
)

In [ ]:
ds

In [ ]:
ds

In [ ]:
from sklearn.neighbors import KDTree

In [ ]:
df = ds[["latitude", "longitude"]].to_dataframe()

In [ ]:
kd_tree = KDTree(df)

In [ ]:
distance, index = kd_tree.query([[55, 75]])

In [ ]:
df.iloc[index[0],:]

In [ ]:
ds.interp(coords={"latitude": 55})

In [ ]:
param_filter = {
            "stepType": "instant",
            "typeOfLevel": "heightAboveGround",
            "parameterCategory": 2,
            "level": 10
          }

ds0 = xr.open_dataset(
    "/home/uch/PVForecast/data/etl/.files/raw/nwp/time=20240710T150000Z/HARMONIE_DINI_SF_2024-07-10T150000Z_2024-07-10T150000Z.grib",
    backend_kwargs={
        "filter_by_keys": param_filter,
        "indexpath": "",
    },
)

In [ ]:
import numpy as np
import datetime as dt

In [ ]:
ds["time"].values.astype('datetime64[us]').item().strftime("%Y%m%dT%H%M%SZ")

In [ ]:
ds = (
    ds
    .drop(["number", "step", "time"])
    .rename({
        "heightAboveGround": "altitiude_m",
        "valid_time": "time_utc",
        "u10": "wind_u_m_s",
        "v10": "wind_v_m_s",
        "si10": "wind_speed_m_s",
        "wdir10": "wind_direction_deg",
    })
)

In [ ]:
ds

In [ ]:
param_filter = {
            "stepType": "instant",
            "typeOfLevel": "heightAboveGround",
            "parameterCategory": 2,
            "level": 10
          }

ds = xr.open_mfdataset(
    ".files/*/*.grib",
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": param_filter,
        "indexpath": "",
    },
    concat_dim="valid_time",
    combine="nested"
)

In [ ]:
ds[]

In [ ]:
ds0 = (
    ds
    .drop(["number", "step", "time"])
    .rename({
        "heightAboveGround": "altitiude_m",
        "valid_time": "time_utc",
        "u10": "wind_u_m_s",
        "v10": "wind_v_m_s",
        "si10": "wind_speed_m_s",
        "wdir10": "wind_direction_deg",
    })
)

In [ ]:
ds0

In [ ]:
np.sqrt(6.558245**2 + (-4.988802)**2)

In [ ]:
ds0.sel(x=0, y=0).to_dataframe()

In [ ]:
param_filter = {
            "stepType": "instant",
            "typeOfLevel": "heightAboveGround",
            "parameterCategory": 2,
            "level": 10
          }

ds = xr.open_mfdataset(
    "/home/uch/PVForecast/data/etl/.files/time=20240710T130000Z/HARMONIE_DINI_SF_2024-07-10T120000Z_2024-07-10T130000Z.grib",
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": param_filter,
        "indexpath": "",
    },
)

In [ ]:
ds["time"].values